# Intialization

In [1]:
# import libraries

import pandas as pd
import numpy as np

from pycoingecko import CoinGeckoAPI
from forex_python.converter import CurrencyRates

import requests
import json

## KUCOIN

In [2]:
# read data (2021/5/17 - 2021/6/1)

data = pd.read_csv("HISTORY_Kucoin.csv")

# derive field - total funds

df = data

df["totalFunds"] = df["dealFunds"] + df["fee"]

# data frame overview

df.head()

,orderCreatedAt,id,clientOid,symbol,side,type,stopPrice,price,size,dealSize,dealFunds,averagePrice,fee,feeCurrency,remark,tags,orderStatus,totalFunds
0,2021/6/1 1:00,60b51626c141750006ec7a63,NaN,EWT-USDT,buy,limit,0,9.00,2.220000,0.000000,0.000000,0.00,0.000000,USDT,NaN,NaN,cancel,0.000000
1,2021/5/23 1:03,60a9397cc1417500060c2ef8,NaN,ETH-USDT,buy,limit,0,2200.00,0.022670,0.022670,49.875100,2200.00,0.049875,USDT,NaN,NaN,done,49.924975
2,2021/5/23 1:03,60a9396f93e36600067be71a,NaN,ADA-USDT,buy,limit,0,1.38,36.213800,36.213800,49.975044,1.38,0.049975,USDT,NaN,NaN,done,50.025019
3,2021/5/23 21:46,60aa5c9dc141750006c64be4,NaN,ETH-USDT,buy,limit,0,1900.00,0.052579,0.052579,99.900100,1900.00,0.099900,USDT,NaN,NaN,done,100.000000
4,2021/5/20 22:34,60a673605d560600079aab4a,NaN,ETH-USDT,buy,limit,0,2500.00,0.020000,0.020000,50.000000,2500.00,0.050000,USDT,NaN,NaN,done,50.050000


### Functions

In [3]:
# return crypto average price: kucoin

def crypto_avg_price(symbol):
    
    # filter data set
    
    df = data.loc[(data['orderStatus'].str.match('done') & (data['symbol'].str.startswith(symbol.upper()))) \
         ,['orderCreatedAt','symbol','side','dealSize','dealFunds','averagePrice','fee','feeCurrency','totalFunds']] \
         .sort_values(by='orderCreatedAt', ascending=True)
         
    # average purchase crypto price
    
    return df['totalFunds'].sum()/df['dealSize'].sum()

# return crypto size: kucoin

def crypto_size(symbol):
    
    # filter data set
    
    df = data.loc[(data['orderStatus'].str.match('done') & data['side'].str.match('buy') & (data['symbol'].str.startswith(symbol.upper()))) \
         ,['orderCreatedAt','symbol','side','dealSize','dealFunds','averagePrice','fee','feeCurrency','totalFunds']]\
         .sort_values(by='orderCreatedAt', ascending=True)
    
    # average purchase crypto price
    
    return df['dealSize'].sum()

In [4]:
# load json file

coinlist = requests.get("https://api.coingecko.com/api/v3/coins/list?include_platform=false" \
                , headers = {"accept": "application/json"})

currencylist = requests.get("https://api.coingecko.com/api/v3/simple/supported_vs_currencies" \
                , headers = {"accept": "application/json"})

# check coin gecko symbol return id

def cryptoid_check(symbol):
    for index,row in enumerate(coinlist.json()):
        if symbol.lower() == row['symbol']:
            return row['id']
    return

# check coin gecko currency return currency

def currency_check(currency):
    if currency.lower() in  currencylist.json():
        return currency.lower()
    return

# return crypto price

def crypto_price(symbol,currency):
    cg = CoinGeckoAPI()
    
    cg_id = cryptoid_check(symbol).lower()
    cg_currency = currency_check(currency).lower()
    
    if not cg_id or not cg_currency:
        return print("Invalid Input")
    else:
        #print(symbol.upper()+"-"+currency.upper())
        return cg.get_price(ids=cg_id, vs_currencies=cg_currency)[cg_id][cg_currency]
    
# return crypto price (usd)

def crypto_price_usd(symbol):
    cg = CoinGeckoAPI()
    
    cg_id = cryptoid_check(symbol)
    
    if not cg_id:
        return print("Invalid Input")
    else:
        #print(symbol.upper()+"-"+currency.upper())
        return cg.get_price(ids=cg_id.lower(), vs_currencies='usd')[cg_id]['usd']
    
# return gain / loss $ (usd): kucoin

def crypto_return_usd(symbol):
    return (crypto_price_usd(symbol) / crypto_price_usd('usdt') - crypto_avg_price(symbol)) * crypto_size(symbol)

# return gain / loss % (usd)

def crypto_return_perc_usd(symbol):
    return (crypto_price_usd(symbol) / crypto_price_usd('usdt') / crypto_avg_price(symbol) - 1) * 100

### Calling Functions

In [5]:
# get crypto portfolio summary: kucoin

c = CurrencyRates()
forex_rate = c.get_rate('USD', 'USD') #https://www.tutorialspoint.com/python-get-the-real-time-currency-exchange-rate
total_pur_vol = total_cur_vol = 0

for symbol in ['BTC','ETH','ADA','KCS']:
    
    pur_vol = cur_vol = 0
    
    pur_vol = crypto_size(symbol)*crypto_avg_price(symbol)*forex_rate
    cur_vol = crypto_size(symbol)*crypto_price_usd(symbol)*forex_rate
    
    print("")
    print(symbol)
    print("Average Purchased Price ($): {:,}".format(round(crypto_avg_price(symbol)/(crypto_price_usd('usdt'))*forex_rate,2)))
    print("Current Price ($): {:,}".format(round(crypto_price_usd(symbol)*forex_rate,2)))
    print("Purchased Volume: {:,}".format(round(pur_vol,2)))
    print("Current Volume: {:,}".format(round(cur_vol,2)))
    print("P/L ($): {:,}".format(round(crypto_return_usd(symbol)*forex_rate,2)))
    print("P/L (%): {}".format(round(crypto_return_perc_usd(symbol),2)))
    
    total_pur_vol += pur_vol
    total_cur_vol += cur_vol
    
print("")    
print("Total Purchased Volume: {}".format(round(total_pur_vol,2)))
print("Total Current Volume: {}".format(round(total_cur_vol,2)))
print("Total P/L ($): {}".format(round(total_cur_vol-total_pur_vol,2)))
print("Total P/L (%): {}".format(round(total_cur_vol/total_pur_vol-1,2)))


BTC
Average Purchased Price ($): 37,645.69
Current Price ($): 46,438.0
Purchased Volume: 200.0
Current Volume: 246.71
P/L ($): 46.71
P/L (%): 23.36

ETH
Average Purchased Price ($): 2,250.01
Current Price ($): 3,446.86
Purchased Volume: 249.97
Current Volume: 382.94
P/L ($): 132.97
P/L (%): 53.19

ADA
Average Purchased Price ($): 1.38
Current Price ($): 2.61
Purchased Volume: 50.03
Current Volume: 94.52
P/L ($): 44.49
P/L (%): 88.94

KCS
Average Purchased Price ($): 12.58
Current Price ($): 12.27
Purchased Volume: 200.1
Current Volume: 195.13
P/L ($): -4.97
P/L (%): -2.48

Total Purchased Volume: 700.1
Total Current Volume: 919.3
Total P/L ($): 219.2
Total P/L (%): 0.31
